<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [ ]:
# Set your student number and name
student_number = 99105429
Name = Amirhossein
Last_Name = Khoshbakht

# Libraries

In [62]:
import csv
import numpy as np
import pandas as pd

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [63]:
class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list, emission_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

    def viterbi(self, sequence: str) -> tuple:
        n_states = len(self.states)
        T = len(sequence)

        viterbi_table = [[0 for _ in range(T)] for _ in range(n_states)]
        backpointer = [[0 for _ in range(T)] for _ in range(n_states)]

        for s in range(n_states):
            viterbi_table[s][0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]
            backpointer[s][0] = 0

        for t in range(1, T):
            for s in range(n_states):
                max_tr_prob = viterbi_table[0][t-1] * self.transition_probabilities[0][s]
                prev_st_selected = 0
                for prev_st in range(1, n_states):
                    tr_prob = viterbi_table[prev_st][t-1] * self.transition_probabilities[prev_st][s]
                    if tr_prob > max_tr_prob:
                        max_tr_prob = tr_prob
                        prev_st_selected = prev_st
                max_prob = max_tr_prob * self.emission_probabilities[s][self.emissions.index(sequence[t])]
                viterbi_table[s][t] = max_prob
                backpointer[s][t] = prev_st_selected

        max_prob = max(value[-1] for value in viterbi_table)
        opt = []
        previous = None

        for s in range(n_states):
            if viterbi_table[s][T-1] == max_prob:
                opt.append(s)
                previous = s
                break

        for t in range(T-1, 0, -1):
            opt.insert(0, backpointer[previous][t])
            previous = backpointer[previous][t]

        path = [self.state_labels[i] for i in opt]
        return path, max_prob

    def forward_algorithm(self, sequence: str) -> list:
        n_states = len(self.states)
        T = len(sequence)

        forward_table = [[0 for _ in range(T)] for _ in range(n_states)]

        for s in range(n_states):
            forward_table[s][0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]

        for t in range(1, T):
            for s in range(n_states):
                forward_table[s][t] = sum(forward_table[prev_st][t-1] * self.transition_probabilities[prev_st][s] for prev_st in range(n_states)) * self.emission_probabilities[s][self.emissions.index(sequence[t])]

        probability_last_states = [forward_table[s][T-1] for s in range(n_states)]
        total_probability = sum(probability_last_states)

        return [p / total_probability for p in probability_last_states]




GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

![Probabilities](Probabilities.png)

In [64]:
states = ['H', 'L']
emissions = ['A', 'C', 'G', 'T']
start_probabilities = [0.5, 0.5]
transition_probabilities = [
    [0.5, 0.5],
    [0.4, 0.6]
]
emission_probabilities = [
    [0.2, 0.3, 0.3, 0.2],
    [0.3, 0.2, 0.2, 0.3]
]


hmm = HMM(states, emissions, start_probabilities, transition_probabilities, emission_probabilities)


sequence = "ATGCGCGATCGATCGAATCGCGT"


path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", ''.join(path))
print("Max Probability (Viterbi):", max_probability)


probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", probability)


Viterbi Path: LLHHHHHLLHHLLHHLLLHHHHL
Max Probability (Viterbi): 1.1438396227480495e-19
Probability (Forward Algorithm): [0.35818008853498684, 0.6418199114650132]


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


In [65]:
class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list, emission_probabilities: list):
        self.states = states
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

    def forward_algorithm(self, sequence: str) -> list:
        n_states = len(self.states)
        T = len(sequence)

        forward_table = [[0 for _ in range(T)] for _ in range(n_states)]

        for s in range(n_states):
            forward_table[s][0] = self.start_probabilities[s] * self.emission_probabilities[s][sequence[0]]

        for t in range(1, T):
            for s in range(n_states):
                forward_table[s][t] = sum(forward_table[prev_st][t-1] * self.transition_probabilities[prev_st][s] for prev_st in range(n_states)) * self.emission_probabilities[s][sequence[t]]

        probability_last_states = [forward_table[s][T-1] for s in range(n_states)]
        total_probability = sum(probability_last_states)

        return [p / total_probability for p in probability_last_states]

def probabilities_from_sequence(sequence: str) -> tuple:
    '''
    This function, `probabilities_from_sequence(sequence)`, computes the probabilities associated with attributes such as blood type, height, weight, and the presence or absence of Dilated Cardiomyopathy (DCM) based on an input DNA sequence.

    ### Input:
    - `sequence`: The observed DNA sequence for which probabilities are to be computed.

    ### Output:
    The function returns dictionaries containing the probabilities for each possible state of the respective attributes:
    - `height_probability_dic`: Probabilities for height categories (160 cm, 170 cm, 180 cm, 190 cm).
    - `weight_probability_dic`: Probabilities for weight categories (50 kg, 65 kg, 80 kg, 95 kg).
    - `blood_type_probability_dic`: Probabilities for blood types (O, A, B, AB).
    - `DCM_probability_dic`: Probabilities for the presence (Y) or absence (N) of Dilated Cardiomyopathy (DCM).
    '''
    height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
                        {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]
    weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
                        {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]
    blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
                            {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]
    DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]

    height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]

    height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
    weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
    blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
    DCM_start_probabilities = [0.25, 0.75]

    height_hmm = HMM(states=['160 cm', '170 cm', '180 cm', '190 cm'], emissions=['H', 'L'],
                     start_probabilities=height_start_probabilities,
                     transition_probabilities=height_transition_probabilities,
                     emission_probabilities=[{'H': height_emissions[0]['H'], 'L': height_emissions[0]['L']},
                                             {'H': height_emissions[1]['H'], 'L': height_emissions[1]['L']},
                                             {'H': height_emissions[2]['H'], 'L': height_emissions[2]['L']},
                                             {'H': height_emissions[3]['H'], 'L': height_emissions[3]['L']}])

    weight_hmm = HMM(states=['50 kg', '65 kg', '80 kg', '95 kg'], emissions=['H', 'L'],
                     start_probabilities=weight_start_probabilities,
                     transition_probabilities=weight_transition_probabilities,
                     emission_probabilities=[{'H': weight_emissions[0]['H'], 'L': weight_emissions[0]['L']},
                                             {'H': weight_emissions[1]['H'], 'L': weight_emissions[1]['L']},
                                             {'H': weight_emissions[2]['H'], 'L': weight_emissions[2]['L']},
                                             {'H': weight_emissions[3]['H'], 'L': weight_emissions[3]['L']}])

    blood_type_hmm = HMM(states=['O', 'A', 'B', 'AB'], emissions=['H', 'L'],
                         start_probabilities=blood_type_start_probabilities,
                         transition_probabilities=blood_type_transition_probabilities,
                         emission_probabilities=[{'H': blood_type_emissions[0]['H'], 'L': blood_type_emissions[0]['L']},
                                                 {'H': blood_type_emissions[1]['H'], 'L': blood_type_emissions[1]['L']},
                                                 {'H': blood_type_emissions[2]['H'], 'L': blood_type_emissions[2]['L']},
                                                 {'H': blood_type_emissions[3]['H'], 'L': blood_type_emissions[3]['L']}])

    DCM_hmm = HMM(states=['Y', 'N'], emissions=['H', 'L'],
                  start_probabilities=DCM_start_probabilities,
                  transition_probabilities=DCM_transition_probabilities,
                  emission_probabilities=[{'H': DCM_emissions[0]['H'], 'L': DCM_emissions[0]['L']},
                                          {'H': DCM_emissions[1]['H'], 'L': DCM_emissions[1]['L']}])

    height_probabilities = height_hmm.forward_algorithm(sequence)
    weight_probabilities = weight_hmm.forward_algorithm(sequence)
    blood_type_probabilities = blood_type_hmm.forward_algorithm(sequence)
    DCM_probabilities = DCM_hmm.forward_algorithm(sequence)

    height_probability_dic = {'160 cm': height_probabilities[0], '170 cm': height_probabilities[1],
                              '180 cm': height_probabilities[2], '190 cm': height_probabilities[3]}
    weight_probability_dic = {'50 kg': weight_probabilities[0], '65 kg': weight_probabilities[1],
                              '80 kg': weight_probabilities[2], '95 kg': weight_probabilities[3]}
    blood_type_probability_dic = {'O': blood_type_probabilities[0], 'A': blood_type_probabilities[1],
                                  'B': blood_type_probabilities[2], 'AB': blood_type_probabilities[3]}
    DCM_probability_dic = {'Y': DCM_probabilities[0], 'N': DCM_probabilities[1]}

    return height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic



In [66]:

sequence = "HHLLHHHLLHLLLLHHLL"

height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_probability)
print("Probability of weight given sequence:", weight_probability)
print("Probability of blood type given sequence:", blood_type_probability)
print("Probability of DCM given sequence:", DCM_probability)


Probability of height given sequence: {'160 cm': 0.3660593174871113, '170 cm': 0.5760951509787053, '180 cm': 0.01585035336762777, '190 cm': 0.041995178166555486}
Probability of weight given sequence: {'50 kg': 0.2845952908892867, '65 kg': 0.42707075390249144, '80 kg': 0.20957356322963913, '95 kg': 0.07876039197858278}
Probability of blood type given sequence: {'O': 0.2962828473309036, 'A': 0.3305080573190477, 'B': 0.08789883086305703, 'AB': 0.2853102644869916}
Probability of DCM given sequence: {'Y': 0.3297758181008331, 'N': 0.670224181899167}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

![Bayesian Net](BN.png)

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [67]:
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [68]:
import numpy as np

def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    '''
    This function, `calculate_sex_probability(height_prob, weight_prob)`, computes the probability of sex based on the probabilities of height and weight.

    Parameters:
    - height_prob: Dictionary containing probabilities for height categories.
    - weight_prob: Dictionary containing probabilities for weight categories.

    Returns:
    - Dictionary containing probabilities for male (M) and female (F) sexes.
    '''

    sex_dic = {'M': 0, 'F': 1}
    sex_probability = np.zeros(2)

    for height, p_height in height_prob.items():
        for weight, p_weight in weight_prob.items():
            for sex in ['M', 'F']:
                sex_probability[sex_dic[sex]] += P_sex_given_hw[(sex, height, weight)] * p_height * p_weight

    return {'M': sex_probability[0], 'F': sex_probability[1]}



In [69]:
P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}


sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.6355000000000002, 'F': 0.36450000000000005}


In [71]:
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [70]:
import numpy as np

def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    '''
    Function: Calculate Heart Disease Probability

    This function, `calculate_heart_disease_probability(DCM_prob, sex_prob, blood_type_prob)`, computes the probability of heart disease based on the probabilities of Dilated Cardiomyopathy (DCM), sex, and blood type.

    Parameters:
    - DCM_prob: Dictionary containing probabilities for DCM categories.
    - sex_prob: Dictionary containing probabilities for sex categories.
    - blood_type_prob: Dictionary containing probabilities for blood type categories.

    Returns:
    - Dictionary containing probabilities for the presence (Y) and absence (N) of heart disease.
    '''

    heart_disease_dic = {'Y': 0, 'N': 1}

    heart_disease_probability = np.zeros(2)

    for DCM in DCM_prob:
        for sex in sex_prob:
            for blood_type in blood_type_prob:
                for heart_disease in ['Y', 'N']:
                    p_hd = P_heart_disease_given_parents[(heart_disease, sex, blood_type, DCM)]
                    heart_disease_probability[heart_disease_dic[heart_disease]] += p_hd * DCM_prob[DCM] * sex_prob[sex] * blood_type_prob[blood_type]

    return {'Y': heart_disease_probability[0], 'N': heart_disease_probability[1]}


In [72]:

P_DCM = {'Y': 0.2, 'N': 0.8}
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
P_sex = {'M': 0.3, 'F': 0.7}


heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ",heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.4152, 'N': 0.5848}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [73]:
df = pd.read_csv('DNA_patients.csv')

df.head()

ID                                                DNA
0   1  CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1   2  TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2   3  ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3   4  TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4   5  AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...

In [74]:
import pandas as pd
import numpy as np
import csv

class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list, emission_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities

    def viterbi(self, sequence: str) -> tuple:
        n_states = len(self.states)
        T = len(sequence)

        viterbi_table = [[0 for _ in range(T)] for _ in range(n_states)]
        backpointer = [[0 for _ in range(T)] for _ in range(n_states)]

        for s in range(n_states):
            viterbi_table[s][0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]
            backpointer[s][0] = 0

        for t in range(1, T):
            for s in range(n_states):
                max_tr_prob = viterbi_table[0][t-1] * self.transition_probabilities[0][s]
                prev_st_selected = 0
                for prev_st in range(1, n_states):
                    tr_prob = viterbi_table[prev_st][t-1] * self.transition_probabilities[prev_st][s]
                    if tr_prob > max_tr_prob:
                        max_tr_prob = tr_prob
                        prev_st_selected = prev_st
                max_prob = max_tr_prob * self.emission_probabilities[s][self.emissions.index(sequence[t])]
                viterbi_table[s][t] = max_prob
                backpointer[s][t] = prev_st_selected

        max_prob = max(value[-1] for value in viterbi_table)
        opt = []
        previous = None

        for s in range(n_states):
            if viterbi_table[s][T-1] == max_prob:
                opt.append(s)
                previous = s
                break

        for t in range(T-1, 0, -1):
            opt.insert(0, backpointer[previous][t])
            previous = backpointer[previous][t]

        path = [self.state_labels[i] for i in opt]
        return path, max_prob

    def forward_algorithm(self, sequence: str) -> list:
        n_states = len(self.states)
        T = len(sequence)

        forward_table = [[0 for _ in range(T)] for _ in range(n_states)]

        for s in range(n_states):
            forward_table[s][0] = self.start_probabilities[s] * self.emission_probabilities[s][self.emissions.index(sequence[0])]

        for t in range(1, T):
            for s in range(n_states):
                forward_table[s][t] = sum(forward_table[prev_st][t-1] * self.transition_probabilities[prev_st][s] for prev_st in range(n_states)) * self.emission_probabilities[s][self.emissions.index(sequence[t])]

        probability_last_states = [forward_table[s][T-1] for s in range(n_states)]
        total_probability = sum(probability_last_states)

        return [p / total_probability for p in probability_last_states]

def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    heart_disease_dic = {'Y': 0, 'N': 1}
    heart_disease_probability = np.zeros(2)

    for DCM in DCM_prob:
        for sex in sex_prob:
            for blood_type in blood_type_prob:
                for heart_disease in ['Y', 'N']:
                    p_hd = P_heart_disease_given_parents[(heart_disease, sex, blood_type, DCM)]
                    heart_disease_probability[heart_disease_dic[heart_disease]] += p_hd * DCM_prob[DCM] * sex_prob[sex] * blood_type_prob[blood_type]

    return {'Y': heart_disease_probability[0], 'N': heart_disease_probability[1]}

df = pd.read_csv('DNA_patients.csv')

DNAs = df['DNA']
states = ['H', 'L']
emissions = ['A', 'C', 'G', 'T']
start_probabilities = [0.5, 0.5]
transition_probabilities = [
    [0.5, 0.5],
    [0.4, 0.6]
]
emission_probabilities = [
    [0.2, 0.3, 0.3, 0.2],
    [0.3, 0.2, 0.2, 0.3]
]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities, emission_probabilities)

GC_content_seqs = []

for seq in DNAs:
    path, max_probability = hmm.viterbi(seq)
    GC_content_seqs.append(path)

P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}
P_DCM = {'Y': 0.2, 'N': 0.8}
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

def calculate_individual_probabilities(gc_content_sequence):
    count_h = gc_content_sequence.count('H')
    count_l = gc_content_sequence.count('L')
    total = count_h + count_l

    proportion_h = count_h / total
    proportion_l = count_l / total

    height_prob = {
        160: height_emissions[0]['H'] * proportion_h + height_emissions[0]['L'] * proportion_l,
        170: height_emissions[1]['H'] * proportion_h + height_emissions[1]['L'] * proportion_l,
        180: height_emissions[2]['H'] * proportion_h + height_emissions[2]['L'] * proportion_l,
        190: height_emissions[3]['H'] * proportion_h + height_emissions[3]['L'] * proportion_l
    }
    weight_prob = {
        50: weight_emissions[0]['H'] * proportion_h + weight_emissions[0]['L'] * proportion_l,
        65: weight_emissions[1]['H'] * proportion_h + weight_emissions[1]['L'] * proportion_l,
        80: weight_emissions[2]['H'] * proportion_h + weight_emissions[2]['L'] * proportion_l,
        95: weight_emissions[3]['H'] * proportion_h + weight_emissions[3]['L'] * proportion_l
    }

    height_sum = sum(height_prob.values())
    weight_sum = sum(weight_prob.values())
    height_prob = {k: v / height_sum for k, v in height_prob.items()}
    weight_prob = {k: v / weight_sum for k, v in weight_prob.items()}

    sex_prob = calculate_sex_probability(height_prob, weight_prob)

    return height_prob, weight_prob, sex_prob

height_emissions = [
    {'H': 0.2, 'L': 0.8},
    {'H': 0.3, 'L': 0.7},
    {'H': 0.9, 'L': 0.1},
    {'H': 0.8, 'L': 0.2}
]
weight_emissions = [
    {'H': 0.3, 'L': 0.7},
    {'H': 0.1, 'L': 0.9},
    {'H': 0.2, 'L': 0.8},
    {'H': 0.7, 'L': 0.3}
]

def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    sex_dic = {'M': 0, 'F': 1}
    sex_probability = np.zeros(2)

    for height, p_height in height_prob.items():
        for weight, p_weight in weight_prob.items():
            for sex in ['M', 'F']:
                sex_probability[sex_dic[sex]] += P_sex_given_hw[(sex, height, weight)] * p_height * p_weight

    return {'M': sex_probability[0], 'F': sex_probability[1]}

heart_disease_classified = []



for idx, seq in enumerate(GC_content_seqs):
    height_prob, weight_prob, sex_prob = calculate_individual_probabilities(seq)

    heart_disease_prob = calculate_heart_disease_probability(P_DCM, sex_prob, P_blood_type)

    heart_disease_classified.append(1 if heart_disease_prob['Y'] > 0.5 else 0)



In [75]:
labels = [(i+1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**